In [1]:
import numpy as np
import pandas as pd
delimiters = [',', ';', '|']
for delimiter in delimiters:
    df = pd.read_csv('proj2_data.csv', delimiter=delimiter, decimal='.', thousands=None)
    if df.shape[1] > 1:
        correct_delimiter = delimiter
        break 

In [2]:
#jezeli nie przecinek
i = 1
for dtype in df.dtypes.items():
    if('float' in str(dtype[1])):
        break
    i += 1
if(i==df.shape[0]):
    df = pd.read_csv('proj2_data.csv', delimiter=correct_delimiter, decimal=',', thousands=None)

In [3]:
#for column in df.columns:
   # if pd.api.types.is_float_dtype(df[column]):
      #  df[column] = df[column].astype('float64')

In [4]:
df.head(13)

,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,python,wej,3.1,2.0,4.4,3.166667,dostateczny,"3,5 pts",dostateczny
1,Nash Wyatt,racing,java,sfe,4.2,2.0,2.0,2.733333,bardzo dobry,5 p,mierny
2,Jadiel Ramirez,media,cplusplus,vaw,4.0,4.9,3.0,3.966667,dobry,3.5,mierny
3,Makaila Atkins,racing,swift,ugt,4.1,5.0,4.8,4.633333,dobry,2,dostateczny
4,Melanie Fuller,racing,python,owb,2.7,2.0,2.0,2.233333,bardzo dobry,pts 2,mierny
5,Layla Woodard,drones,python,wuf,2.8,2.0,2.0,2.266667,bardzo dobry,"3,5p",dostateczny
6,Edwin Hale,media,swift,ceq,4.9,3.0,3.4,3.766667,dostateczny,-1 (dnf),niedostateczny
7,Travis Rocha,robotics,cplusplus,eir,5.0,4.9,5.0,4.966667,dostateczny,0,niedostateczny
8,Ricky Howell,media,java,vos,2.1,2.0,2.0,2.033333,dobry,waiting,niedostateczny
9,Addyson Brennan,drones,java,cov,4.4,3.7,4.0,4.033333,bardzo dobry,-,niedostateczny


In [5]:
import pickle
df.to_pickle('proj2_ex01.pkl')

In [6]:
file1 = open('proj2_scale.txt', 'r')
Lines = file1.read().splitlines()

In [7]:
print(Lines)

['niedostateczny', 'mierny', 'dostateczny', 'dobry', 'bardzo dobry']


In [8]:
scale = {}
j = 1
for line in Lines:
    scale[line] = j
    j += 1
print(scale)

{'niedostateczny': 1, 'mierny': 2, 'dostateczny': 3, 'dobry': 4, 'bardzo dobry': 5}


In [9]:
df2 = df.copy()
columns_with_elements = df2.columns[df2.isin(Lines).any()]

for column in columns_with_elements:
    df2[column] = df2[column].map(scale)

In [10]:
df2.to_pickle('proj2_ex02.pkl')

In [11]:
df3 = df.copy()
for column in columns_with_elements:
    df3[column] = df3[column].astype('category')
    df3[column] = df3[column].cat.set_categories(Lines)

In [12]:
for column in columns_with_elements:
    print(df3[column])
df3.to_pickle('proj2_ex03.pkl')

0      dostateczny
1     bardzo dobry
2            dobry
3            dobry
4     bardzo dobry
5     bardzo dobry
6      dostateczny
7      dostateczny
8            dobry
9     bardzo dobry
10     dostateczny
11    bardzo dobry
Name: task_grade, dtype: category
Categories (5, object): ['niedostateczny', 'mierny', 'dostateczny', 'dobry', 'bardzo dobry']
0        dostateczny
1             mierny
2             mierny
3        dostateczny
4             mierny
5        dostateczny
6     niedostateczny
7     niedostateczny
8     niedostateczny
9     niedostateczny
10       dostateczny
11            mierny
Name: final_grade, dtype: category
Categories (5, object): ['niedostateczny', 'mierny', 'dostateczny', 'dobry', 'bardzo dobry']


In [15]:
import re

dfreg = pd.DataFrame()
regex = r'[-]?[0-9]+[\.,]?[0-9]*'
columns_with_possible_numbers = []
for column in df.columns:
    if(pd.api.types.is_numeric_dtype(df[column]) is False and df[column].str.contains(regex).any()):
        columns_with_possible_numbers.append(column)
for col in columns_with_possible_numbers:
    lista = df[col].tolist()
    for k in range (0, len(lista)):
        znal = str(re.findall(regex, lista[k])).strip('[\']').replace(',','.')
        if(znal == ''):
            lista[k] = None
        else:
            lista[k] = float(znal)
    dfreg[col] = lista
        
print(columns_with_possible_numbers)

dfreg.to_pickle('proj2_ex04.pkl')
dfreg.head(12)


['jury_score']


,jury_score
0,3.5
1,5.0
2,3.5
3,2.0
4,2.0
5,3.5
6,-1.0
7,0.0
8,NaN
9,NaN


In [14]:
wanted = []
i = 1
for column in df.columns:
    if(not pd.api.types.is_numeric_dtype(df[column]) and df[column].nunique() <= 10 and df[column].str.isalpha().all() and df[column].str.islower().all() and not df[column].isin(Lines).any()):
       wanted.append(column)
for w in wanted:
    dummy = pd.get_dummies(df[w])
    filename = f'proj2_ex05_{i}.pkl'
    dummy.to_pickle(filename)
    i += 1